In [1]:
# %pip install langchain_google_genai

In [2]:
# %pip install langchain_huggingface

In [3]:
# %pip install FAISS-cpu

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [5]:
transcript_list=[{'text': 'Imagine you happen across a short movie script that',
  'start': 1.14,
  'duration': 2.836},
 {'text': 'describes a scene between a person and their AI assistant.',
  'start': 3.976,
  'duration': 3.164},
 {'text': "The script has what the person asks the AI, but the AI's response has been torn off.",
  'start': 7.48,
  'duration': 5.58},
 {'text': 'Suppose you also have this powerful magical machine that can take',
  'start': 13.06,
  'duration': 3.92},
 {'text': 'any text and provide a sensible prediction of what word comes next.',
  'start': 16.98,
  'duration': 3.98},
 {'text': 'You could then finish the script by feeding in what you have to the machine,',
  'start': 21.5,
  'duration': 4.006},
 {'text': "seeing what it would predict to start the AI's answer,",
  'start': 25.506,
  'duration': 2.862},
 {'text': 'and then repeating this over and over with a growing script completing the dialogue.',
  'start': 28.368,
  'duration': 4.372},
 {'text': "When you interact with a chatbot, this is exactly what's happening.",
  'start': 33.38,
  'duration': 3.1},
 {'text': 'A large language model is a sophisticated mathematical function',
  'start': 37.02,
  'duration': 3.681},
 {'text': 'that predicts what word comes next for any piece of text.',
  'start': 40.701,
  'duration': 3.279},
 {'text': 'Instead of predicting one word with certainty, though,',
  'start': 44.38,
  'duration': 3.022},
 {'text': 'what it does is assign a probability to all possible next words.',
  'start': 47.402,
  'duration': 3.518},
 {'text': 'To build a chatbot, you lay out some text that describes an interaction between a user',
  'start': 51.62,
  'duration': 5.18},
 {'text': 'and a hypothetical AI assistant, add on whatever the user types in as the first part of',
  'start': 56.8,
  'duration': 5.24},
 {'text': 'the interaction, and then have the model repeatedly predict the next word that such a',
  'start': 62.04,
  'duration': 5.12},
 {'text': "hypothetical AI assistant would say in response, and that's what's presented to the user.",
  'start': 67.16,
  'duration': 5.3},
 {'text': 'In doing this, the output tends to look a lot more natural if',
  'start': 73.08,
  'duration': 3.134},
 {'text': 'you allow it to select less likely words along the way at random.',
  'start': 76.214,
  'duration': 3.286},
 {'text': 'So what this means is even though the model itself is deterministic,',
  'start': 80.14,
  'duration': 3.48},
 {'text': "a given prompt typically gives a different answer each time it's run.",
  'start': 83.62,
  'duration': 3.48},
 {'text': 'Models learn how to make these predictions by processing an enormous amount of text,',
  'start': 88.04,
  'duration': 4.292},
 {'text': 'typically pulled from the internet.',
  'start': 92.332,
  'duration': 1.768},
 {'text': 'For a standard human to read the amount of text that was used to train GPT-3,',
  'start': 94.1,
  'duration': 5.371},
 {'text': 'for example, if they read non-stop 24-7, it would take over 2600 years.',
  'start': 99.471,
  'duration': 4.889},
 {'text': 'Larger models since then train on much, much more.',
  'start': 104.72,
  'duration': 2.62},
 {'text': 'You can think of training a little bit like tuning the dials on a big machine.',
  'start': 108.2,
  'duration': 3.58},
 {'text': 'The way that a language model behaves is entirely determined by these',
  'start': 112.28,
  'duration': 4.021},
 {'text': 'many different continuous values, usually called parameters or weights.',
  'start': 116.301,
  'duration': 4.079},
 {'text': 'Changing those parameters will change the probabilities',
  'start': 121.02,
  'duration': 3.079},
 {'text': 'that the model gives for the next word on a given input.',
  'start': 124.099,
  'duration': 3.081},
 {'text': 'What puts the large in large language model is how',
  'start': 127.86,
  'duration': 2.867},
 {'text': 'they can have hundreds of billions of these parameters.',
  'start': 130.727,
  'duration': 3.093},
 {'text': 'No human ever deliberately sets those parameters.',
  'start': 135.2,
  'duration': 2.84},
 {'text': 'Instead, they begin at random, meaning the model just outputs gibberish,',
  'start': 138.44,
  'duration': 4.203},
 {'text': "but they're repeatedly refined based on many example pieces of text.",
  'start': 142.643,
  'duration': 3.917},
 {'text': 'One of these training examples could be just a handful of words,',
  'start': 147.14,
  'duration': 3.516},
 {'text': 'or it could be thousands, but in either case, the way this works is to',
  'start': 150.656,
  'duration': 3.84},
 {'text': 'pass in all but the last word from that example into the model and',
  'start': 154.496,
  'duration': 3.624},
 {'text': 'compare the prediction that it makes with the true last word from the example.',
  'start': 158.12,
  'duration': 4.22},
 {'text': 'An algorithm called backpropagation is used to tweak all of the parameters',
  'start': 163.26,
  'duration': 4.133},
 {'text': 'in such a way that it makes the model a little more likely to choose',
  'start': 167.393,
  'duration': 3.803},
 {'text': 'the true last word and a little less likely to choose all the others.',
  'start': 171.196,
  'duration': 3.804},
 {'text': 'When you do this for many, many trillions of examples,',
  'start': 175.74,
  'duration': 3.01},
 {'text': 'not only does the model start to give more accurate predictions on the training data,',
  'start': 178.75,
  'duration': 4.708},
 {'text': "but it also starts to make more reasonable predictions on text that it's never",
  'start': 183.458,
  'duration': 4.325},
 {'text': 'seen before.', 'start': 187.783, 'duration': 0.657},
 {'text': 'Given the huge number of parameters and the enormous amount of training data,',
  'start': 189.42,
  'duration': 4.499},
 {'text': 'the scale of computation involved in training a large language model is mind-boggling.',
  'start': 193.919,
  'duration': 4.961},
 {'text': 'To illustrate, imagine that you could perform one',
  'start': 199.6,
  'duration': 2.685},
 {'text': 'billion additions and multiplications every single second.',
  'start': 202.285,
  'duration': 3.115},
 {'text': 'How long do you think it would take for you to do all of the',
  'start': 206.06,
  'duration': 3.266},
 {'text': 'operations involved in training the largest language models?',
  'start': 209.326,
  'duration': 3.214},
 {'text': 'Do you think it would take a year?',
  'start': 213.46,
  'duration': 1.579},
 {'text': 'Maybe something like 10,000 years?',
  'start': 216.039,
  'duration': 1.921},
 {'text': 'The answer is actually much more than that.',
  'start': 219.02,
  'duration': 1.78},
 {'text': "It's well over 100 million years.",
  'start': 221.12,
  'duration': 2.78},
 {'text': 'This is only part of the story, though.',
  'start': 225.52,
  'duration': 1.84},
 {'text': 'This whole process is called pre-training.',
  'start': 227.54,
  'duration': 1.68},
 {'text': 'The goal of auto-completing a random passage of text from the',
  'start': 229.5,
  'duration': 3.146},
 {'text': 'internet is very different from the goal of being a good AI assistant.',
  'start': 232.646,
  'duration': 3.554},
 {'text': 'To address this, chatbots undergo another type of training,',
  'start': 236.88,
  'duration': 3.2},
 {'text': 'just as important, called reinforcement learning with human feedback.',
  'start': 240.08,
  'duration': 3.68},
 {'text': 'Workers flag unhelpful or problematic predictions,',
  'start': 244.48,
  'duration': 3.018},
 {'text': "and their corrections further change the model's parameters,",
  'start': 247.498,
  'duration': 3.611},
 {'text': 'making them more likely to give predictions that users prefer.',
  'start': 251.109,
  'duration': 3.671},
 {'text': 'Looking back at the pre-training, though, this staggering amount of',
  'start': 254.78,
  'duration': 4.08},
 {'text': 'computation is only made possible by using special computer chips that',
  'start': 258.86,
  'duration': 4.26},
 {'text': 'are optimized for running many operations in parallel, known as GPUs.',
  'start': 263.12,
  'duration': 4.14},
 {'text': 'However, not all language models can be easily parallelized.',
  'start': 268.12,
  'duration': 3.5},
 {'text': 'Prior to 2017, most language models would process text one word at a time,',
  'start': 272.08,
  'duration': 4.737},
 {'text': 'but then a team of researchers at Google introduced a new model known as the transformer.',
  'start': 276.817,
  'duration': 5.623},
 {'text': "Transformers don't read text from the start to the finish,",
  'start': 283.3,
  'duration': 3.445},
 {'text': 'they soak it all in at once, in parallel.',
  'start': 286.745,
  'duration': 2.395},
 {'text': 'The very first step inside a transformer, and most other language models for that matter,',
  'start': 289.9,
  'duration': 4.7},
 {'text': 'is to associate each word with a long list of numbers.',
  'start': 294.6,
  'duration': 2.82},
 {'text': 'The reason for this is that the training process only works with continuous values,',
  'start': 297.86,
  'duration': 4.536},
 {'text': 'so you have to somehow encode language using numbers,',
  'start': 302.396,
  'duration': 2.916},
 {'text': 'and each of these lists of numbers may somehow encode the meaning of the',
  'start': 305.312,
  'duration': 3.942},
 {'text': 'corresponding word.', 'start': 309.254, 'duration': 1.026},
 {'text': 'What makes transformers unique is their reliance',
  'start': 310.28,
  'duration': 3.08},
 {'text': 'on a special operation known as attention.',
  'start': 313.36,
  'duration': 2.64},
 {'text': 'This operation gives all of these lists of numbers a chance to talk to one another',
  'start': 316.98,
  'duration': 4.704},
 {'text': 'and refine the meanings they encode based on the context around, all done in parallel.',
  'start': 321.684,
  'duration': 4.876},
 {'text': 'For example, the numbers encoding the word bank might be changed based on the',
  'start': 327.4,
  'duration': 4.307},
 {'text': 'context surrounding it to somehow encode the more specific notion of a riverbank.',
  'start': 331.707,
  'duration': 4.473},
 {'text': 'Transformers typically also include a second type of operation known',
  'start': 337.28,
  'duration': 3.749},
 {'text': 'as a feed-forward neural network, and this gives the model extra',
  'start': 341.029,
  'duration': 3.532},
 {'text': 'capacity to store more patterns about language learned during training.',
  'start': 344.561,
  'duration': 3.859},
 {'text': 'All of this data repeatedly flows through many different iterations of',
  'start': 349.28,
  'duration': 4.121},
 {'text': 'these two fundamental operations, and as it does so,',
  'start': 353.401,
  'duration': 3.077},
 {'text': 'the hope is that each list of numbers is enriched to encode whatever',
  'start': 356.478,
  'duration': 4.006},
 {'text': 'information might be needed to make an accurate prediction of what word',
  'start': 360.484,
  'duration': 4.18},
 {'text': 'follows in the passage.', 'start': 364.664, 'duration': 1.336},
 {'text': 'At the end, one final function is performed on the last vector in this sequence,',
  'start': 367.0,
  'duration': 4.534},
 {'text': 'which now has had a chance to be influenced by all the other context from the input text,',
  'start': 371.534,
  'duration': 5.039},
 {'text': 'as well as everything the model learned during training,',
  'start': 376.573,
  'duration': 3.191},
 {'text': 'to produce a prediction of the next word.',
  'start': 379.764,
  'duration': 2.296},
 {'text': "Again, the model's prediction looks like a probability for every possible next word.",
  'start': 382.48,
  'duration': 4.88},
 {'text': 'Although researchers design the framework for how each of these steps work,',
  'start': 388.56,
  'duration': 4.234},
 {'text': "it's important to understand that the specific behavior is an emergent phenomenon",
  'start': 392.794,
  'duration': 4.568},
 {'text': 'based on how those hundreds of billions of parameters are tuned during training.',
  'start': 397.362,
  'duration': 4.458},
 {'text': 'This makes it incredibly challenging to determine',
  'start': 402.48,
  'duration': 2.59},
 {'text': 'why the model makes the exact predictions that it does.',
  'start': 405.07,
  'duration': 2.85},
 {'text': 'What you can see is that when you use large language model predictions to autocomplete',
  'start': 408.44,
  'duration': 5.338},
 {'text': 'a prompt, the words that it generates are uncannily fluent, fascinating, and even useful.',
  'start': 413.778,
  'duration': 5.462},
 {'text': "If you're a new viewer and you're curious about more details on how",
  'start': 425.719,
  'duration': 3.107},
 {'text': 'transformers and attention work, boy do I have some material for you.',
  'start': 428.826,
  'duration': 3.153},
 {'text': 'One option is to jump into a series I made about deep learning,',
  'start': 432.399,
  'duration': 3.681},
 {'text': 'where we visualize and motivate the details of attention and all the other steps',
  'start': 436.08,
  'duration': 4.66},
 {'text': 'in a transformer.', 'start': 440.74, 'duration': 0.979},
 {'text': 'Also, on my second channel I just posted a talk I gave a couple',
  'start': 442.099,
  'duration': 3.43},
 {'text': 'months ago about this topic for the company TNG in Munich.',
  'start': 445.529,
  'duration': 3.11},
 {'text': 'Sometimes I actually prefer the content I make as a casual talk rather than a produced',
  'start': 449.079,
  'duration': 4.022},
 {'text': 'video, but I leave it up to you which one of these feels like the better follow-on.',
  'start': 453.101,
  'duration': 3.838}]

In [6]:
# transcript_list

In [7]:
# video_id = "Gfr50f6ZBvo" # only the ID, not full URL
# try:
#     # If you don’t care which language, this returns the “best” one
#     transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

#     # Flatten it to plain text
#     transcript = " ".join(chunk["text"] for chunk in transcript_list)
#     print(transcript)

# except TranscriptsDisabled:
#     print("No captions available for this video.")

In [8]:
transcript=" ".join(chunk["text"] for chunk in transcript_list)


Step 1b - Indexing (Text Splitting)

In [9]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = splitter.create_documents([transcript])

In [10]:
# chucnks[100]
len(chunks)

94

In [11]:
chunks[1]

Document(metadata={}, page_content="person and their AI assistant. The script has what the person asks the AI, but the AI's response")

In [12]:
# %pip install sentence-transformers

In [13]:
embedding=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [14]:
vector_store = FAISS.from_documents(
    chunks, embedding)

In [15]:
vector_store.index_to_docstore_id

{0: 'ca988ba7-a08a-49ee-98e7-3feca8592653',
 1: '4ad3372b-3ec7-4491-9106-64742bc2394b',
 2: '269e3c25-045b-480b-8e88-518a96c42132',
 3: '057be560-551a-4277-8f21-3ce4f220ca52',
 4: '80d81474-71ac-42f4-b94d-be150cf60a30',
 5: '3c53a986-89a4-48fc-9ee7-2e644ce00a6f',
 6: 'c31ab298-e489-47e6-a07d-847cf24455c3',
 7: 'e496b7e2-551c-4716-81c0-28df7a286457',
 8: '1542f870-a31c-4437-b0f5-c3bfea91418b',
 9: '73381485-3a5b-42a4-bdc1-8556f37c8452',
 10: 'f123cb35-f49e-4487-8df8-b39fe7a22fc4',
 11: 'e443e742-5b4e-4b4b-bd67-709303e14a97',
 12: 'f9e43d92-31ba-4358-aa1c-7c1e44c434ae',
 13: '5e46f279-b145-4c9f-91d7-36708cd61f63',
 14: 'dcc84ef2-e508-48ff-b544-2b3f9e261b48',
 15: 'a5a3e6f5-170a-4519-8ad0-60a1b158a178',
 16: 'f6d7f6a8-848d-48d7-b417-6e7b5f033537',
 17: 'e911d60f-4f6c-4a1c-9776-16fe39427533',
 18: 'ddd74f9e-aa92-43ea-a148-da5d819ef75b',
 19: '3a882e9d-3d92-467a-afcc-774b37ad554e',
 20: '5fc0b4b7-68e8-4639-9d56-c201f50fa882',
 21: '182a3499-4db1-4a91-839b-203372faa5df',
 22: '8f44ca69-ce67-

In [16]:
vector_store.get_by_ids(['dc13173a-b0ab-4898-a81d-d8e8e5c7949a'])

[]

In [17]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [18]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000186761FEB50>, search_kwargs={'k': 4})

In [19]:
retriever.invoke('What is deepmind')

[Document(id='09f7e13b-bf46-456b-aaf0-9eae8c884a9f', metadata={}, page_content='a series I made about deep learning, where we visualize and motivate the details of attention and'),
 Document(id='20c40641-e1de-47d9-82fb-8be4dac79e05', metadata={}, page_content='feed-forward neural network, and this gives the model extra capacity to store more patterns about'),
 Document(id='ca988ba7-a08a-49ee-98e7-3feca8592653', metadata={}, page_content='Imagine you happen across a short movie script that describes a scene between a person and their AI'),
 Document(id='f7d3d608-9ac3-421f-b768-969106f1b013', metadata={}, page_content='of text from the internet is very different from the goal of being a good AI assistant. To address')]

Augmentated

In [24]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from dotenv import  load_dotenv
load_dotenv()

True

In [37]:
llms=HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

In [38]:
LLMS=ChatHuggingFace(llm=llms)

In [28]:
# LLMS=ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [29]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [30]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [31]:
retrieved_docs

[Document(id='463aa189-65c6-444f-beb6-1013beaa4da2', metadata={}, page_content='of attention and all the other steps in a transformer. Also, on my second channel I just posted a'),
 Document(id='31e9aceb-85f8-4a08-abaa-aeb3a8bdf32a', metadata={}, page_content='over 100 million years. This is only part of the story, though. This whole process is called'),
 Document(id='ca988ba7-a08a-49ee-98e7-3feca8592653', metadata={}, page_content='Imagine you happen across a short movie script that describes a scene between a person and their AI'),
 Document(id='9c1f9668-cf81-4c90-93ae-537500bb9bf5', metadata={}, page_content="If you're a new viewer and you're curious about more details on how transformers and attention")]

In [32]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"of attention and all the other steps in a transformer. Also, on my second channel I just posted a\n\nover 100 million years. This is only part of the story, though. This whole process is called\n\nImagine you happen across a short movie script that describes a scene between a person and their AI\n\nIf you're a new viewer and you're curious about more details on how transformers and attention"

In [33]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [34]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      of attention and all the other steps in a transformer. Also, on my second channel I just posted a\n\nover 100 million years. This is only part of the story, though. This whole process is called\n\nImagine you happen across a short movie script that describes a scene between a person and their AI\n\nIf you're a new viewer and you're curious about more details on how transformers and attention\n      Question: is the topic of nuclear fusion discussed in this video? if yes then what was discussed\n    ")

Generation

In [ ]:
answer = LLMS.invoke(final_prompt)
print(answer.content)